In [1]:
import pandas as pd
import numpy as np
import os
import requests
import env
import wrangle as w
import explore as exp

In [2]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

In [3]:
# Example usage:
app_token = env.app_token
year_to_retrieve = '2022'
max_req = 2000  # Specify the maximum number of observations to retrieve

In [4]:
df = w.get_data(year_to_retrieve, app_token, max_req)
df = w.date_time(df)
df = w.initial_reorder_cols(df)
df = w.drop_3plus(df)
df = w.single_coll(df)
df = w.motorist_only(df)
df = w.second_drop(df)
df = w.null_zero_loc(df)

CSV file for 2022 already exists. Loading data from the CSV.


In [5]:
combined_three = pd.read_csv('final_geo_filled_errors.csv', index_col=0)

In [6]:
# Remove duplicates in 'df' when all columns have the same information
df.drop_duplicates(keep='first', inplace=True)
combined_three.drop_duplicates(keep='first', inplace=True)

In [7]:
# Create 'df_b' containing rows with matching 'collision_id' in 'combined_three'
df_b = df[df['collision_id'].isin(combined_three['collision_id'])].copy()

# Create 'df_a' containing rows not matched in 'combined_three'
df_a = df[~df['collision_id'].isin(combined_three['collision_id'])].copy()


In [8]:
df_c = df_b.copy()

In [9]:
len(df_a), len(df_b), len(combined_three)

(27113, 27091, 27091)

In [10]:
# Merge the two DataFrames on 'collision_id' and select specific columns
merged_df = df_c.merge(combined_three[['collision_id', 'on_street_name', 'borough', 'zip_code']],
                     on='collision_id', how='left')


In [17]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27091 entries, 0 to 27090
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   crash_datetime                 27091 non-null  datetime64[ns]
 1   crash_date                     27091 non-null  datetime64[ns]
 2   crash_time                     27091 non-null  object        
 3   collision_id                   27091 non-null  int64         
 4   latitude                       27091 non-null  float64       
 5   longitude                      27091 non-null  float64       
 6   location                       27091 non-null  object        
 7   on_street_name_x               15226 non-null  object        
 8   borough_x                      11529 non-null  object        
 9   zip_code_x                     11521 non-null  float64       
 10  number_of_persons_injured      27091 non-null  int64         
 11  number_of_perso

In [18]:
# Drop columns with '_x' suffix
merged_df = merged_df.drop(columns=['on_street_name_x', 'borough_x', 'zip_code_x'])

# Rename columns with '_y' suffix
merged_df = merged_df.rename(columns={'on_street_name_y': 'on_street_name',
                                      'borough_y': 'borough',
                                      'zip_code_y': 'zip_code'})


In [22]:
len(df_a), len(df_b), len(df_c), len(combined_three), len(merged_df)

(27113, 27091, 27091, 27091, 27091)

In [19]:
# Example usage:
filled_df_info = exp.create_info_dataframe(merged_df)  # Assuming data_df is your DataFrame
filled_df_info

,Column,Null_Count,Null_Percentage,Zero_Count,Blank_Count,Unique_Values,Data_Type
0,crash_datetime,0,0.0,0,0,24571,datetime64[ns]
1,crash_date,0,0.0,0,0,365,datetime64[ns]
2,crash_time,0,0.0,0,0,1432,object
3,collision_id,0,0.0,0,0,27091,int64
4,latitude,0,0.0,0,0,16518,float64
5,longitude,0,0.0,0,0,14373,float64
6,location,0,0.0,0,0,18602,object
7,number_of_persons_injured,0,0.0,19243,0,12,int64
8,number_of_persons_killed,0,0.0,27069,0,2,int64
9,number_of_motorist_injured,0,0.0,20005,0,12,int64


In [20]:
merged_df.head()

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,number_of_persons_injured,number_of_persons_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2,on_street_name,borough,zip_code
0,2022-01-01 05:17:00,2022-01-01,05:17:00,4491857,40.746930,-73.84866,"{'latitude': '40.74693', 'longitude': '-73.848...",1,0,1,0,Sedan,Passing or Lane Usage Improper,Sedan,Unsafe Lane Changing,GRAND CENTRAL PKWY,Queens,11368.0
1,2022-01-01 01:30:00,2022-01-01,01:30:00,4491344,40.819157,-73.96038,"{'latitude': '40.819157', 'longitude': '-73.96...",0,0,0,0,Sedan,Unspecified,Station Wagon/Sport Utility Vehicle,NaN,HENRY HUDSON PARKWAY,Manhattan,10027.0
2,2022-01-01 16:40:00,2022-01-01,16:40:00,4491478,40.806107,-73.91799,"{'latitude': '40.806107', 'longitude': '-73.91...",0,0,0,0,Sedan,Unspecified,Sedan,Unspecified,Saint Ann's Avenue,BRONX,10454.0
3,2022-01-01 02:53:00,2022-01-01,02:53:00,4491586,40.646034,-73.99678,"{'latitude': '40.646034', 'longitude': '-73.99...",0,0,0,0,Station Wagon/Sport Utility Vehicle,Unspecified,Station Wagon/Sport Utility Vehicle,Unspecified,40th Street,BROOKLYN,11232.0
4,2022-01-01 17:00:00,2022-01-01,17:00:00,4491660,40.701195,-73.91409,"{'latitude': '40.701195', 'longitude': '-73.91...",0,0,0,0,Sedan,Passing Too Closely,Sedan,Unspecified,WYCKOFF AVENUE,Brooklyn,11237.0


In [25]:
merged_df.to_csv('final_geo_filled.csv')

In [23]:
# Concatenate df_a and merged_df vertically and reindex columns to match df_a
concatenated_df = pd.concat([df_a, merged_df], axis=0).reindex(columns=df_a.columns)

# Reset the index if needed
concatenated_df.reset_index(drop=True, inplace=True)


In [24]:
len(concatenated_df)

54204